In [2]:
import os
from glob import glob
import wfdb
import numpy as np
import pandas as pd
import scipy

In [3]:
root = "/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/Datasets"
new_data_folder = "PTB_XL_data"
datapath = os.path.join(root , new_data_folder)

In [4]:
# os.mkdir("/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/Datasets/Processed_PTB_XL_data")
# os.chdir("/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/Datasets/Processed_PTB_XL_data")
# for file in data_records:
#     data , _ = wfdb.rdsamp(file)
#     dict = {}
#     dict["val"] = data
#     scipy.io.savemat(file_name=file.split('/')[-1] + ".mat" , mdict=dict)

In [5]:
data_records = []

with open(os.path.join(datapath, "RECORDS"), "r") as f:
    for line in f:
        if "records500" in line:
            data_records.append(os.path.join(datapath , line.strip()))

In [6]:
meta_info = pd.read_csv("/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/Datasets/PTB_XL_data/ptbxl_database.csv")
meta_info['scp_codes']


0                 {'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}
1                             {'NORM': 80.0, 'SBRAD': 0.0}
2                               {'NORM': 100.0, 'SR': 0.0}
3                               {'NORM': 100.0, 'SR': 0.0}
4                               {'NORM': 100.0, 'SR': 0.0}
                               ...                        
21794    {'NDT': 100.0, 'PVC': 100.0, 'VCLVH': 0.0, 'ST...
21795             {'NORM': 100.0, 'ABQRS': 0.0, 'SR': 0.0}
21796                           {'ISCAS': 50.0, 'SR': 0.0}
21797                           {'NORM': 100.0, 'SR': 0.0}
21798                           {'NORM': 100.0, 'SR': 0.0}
Name: scp_codes, Length: 21799, dtype: object

In [7]:
scp_codes_df = pd.read_csv("/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/Datasets/PTB_XL_data/scp_statements.csv")
scp_codes_df.rename(columns = {'Unnamed: 0' : 'Potential_diagnosis'} , inplace = True)

In [8]:
print(meta_info.columns)
new_df = meta_info[['filename_hr' , 'ecg_id' , 'age' , 'sex' , 'scp_codes']]

Index(['ecg_id', 'patient_id', 'age', 'sex', 'height', 'weight', 'nurse',
       'site', 'device', 'recording_date', 'report', 'scp_codes', 'heart_axis',
       'infarction_stadium1', 'infarction_stadium2', 'validated_by',
       'second_opinion', 'initial_autogenerated_report', 'validated_by_human',
       'baseline_drift', 'static_noise', 'burst_noise', 'electrodes_problems',
       'extra_beats', 'pacemaker', 'strat_fold', 'filename_lr', 'filename_hr'],
      dtype='object')


In [9]:
# finding the non-zero values in scp_codes_column
collection_non_zero_strings = []
for row in new_df.iterrows():
    non_zero_string = ""
    dict_string = row[1]['scp_codes']
    dict_string_split = dict_string.split(",")
    dict_string_split = [x.split(":") for x in dict_string_split]
    for x in dict_string_split:
        x[0] = x[0].strip(' ')
        x[0] = x[0].strip("{")
        x[0] = x[0].strip(' ')
        x[0] = x[0].strip('\'')
        x[1] = x[1].strip(' ')
        x[1] = x[1].strip('}')
        x[1] = float(x[1])
        if x[1]!=0:
            non_zero_string = non_zero_string + x[0] + " "
    collection_non_zero_strings.append(non_zero_string)

new_df['potential_diagnosis'] = collection_non_zero_strings
new_df.drop(columns = ['scp_codes'] , inplace = True)
split_diagnosis = new_df['potential_diagnosis'].str.split(" " , expand = True)
new_df = pd.concat((new_df , split_diagnosis) , axis = 1)
new_df.drop(columns = ['potential_diagnosis'] , inplace = True)
new_df.rename(columns = {0 : "diagnosis_0" , 1 : "diagnosis_1" , 2 : "diagnosis_2" , 3 : "diagnosis_3" , 4 : "diagnosis_4" , 5 : "diagnosis_5"} , inplace = True)

/var/folders/x0/r7576wrd5w3dfxhxvxn3v81w0000gn/T/ipykernel_2209/2044714244.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['potential_diagnosis'] = collection_non_zero_strings
/var/folders/x0/r7576wrd5w3dfxhxvxn3v81w0000gn/T/ipykernel_2209/2044714244.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.drop(columns = ['scp_codes'] , inplace = True)


In [10]:
new_df.to_csv("/Users/tanmaygoyal/Desktop/Assignments and Events/Biomedicine/ECG_Project/REFERENCE_ptb_xl.csv" , index = False , header = True)

In [11]:
# creating a table for the diagnosis vs number of values
columns = ['diagnosis_' + str(i) for i in range(6)]
unique_diagnosis = new_df['diagnosis_0'].unique().tolist()
for col in columns[1:]:
    unique_diagnosis += new_df[col].unique().tolist()
unique_diagnosis = list(set(unique_diagnosis))
unique_diagnosis.remove('')
dict_counts = {}
for diagnosis in unique_diagnosis:
    dict_counts[diagnosis] = []
    for col in columns:
        try:
            dict_counts[diagnosis].append(new_df[col].value_counts()[diagnosis])
        except:
            dict_counts[diagnosis].append(0)

count_df = pd.DataFrame(dict_counts).T
count_df.rename(columns = {i : "diagnosis_" + str(i) for i in range(6)} , inplace = True)
for col in count_df.columns:
    count_df.loc[None , col] =  len(new_df) - count_df[col].sum()
count_df

,diagnosis_0,diagnosis_1,diagnosis_2,diagnosis_3,diagnosis_4,diagnosis_5
INJAL,36,81,25,3,0,0
CRBBB,254,189,95,3,0,0
NDT,1475,312,36,2,0,0
IPMI,30,3,0,0,0,0
DIG,16,85,54,15,7,0
NORM,9514,0,0,0,0,0
AMI,289,64,0,0,0,0
IRBBB,504,503,95,16,0,0
PVC,14,516,295,149,53,0
STACH,3,1,0,0,0,0


In [12]:
for col in count_df.columns:
    print(col , count_df[col].sum())

diagnosis_0 21799
diagnosis_1 21799
diagnosis_2 21799
diagnosis_3 21799
diagnosis_4 21799
diagnosis_5 21799
